In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import os
import sys
import tarfile
import numpy as np

Using TensorFlow backend.


RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

In [2]:
data_root = os.sep+os.path.join('tmp', 'simpsons')
dest_filename = os.path.join(data_root, 'simpsons_dataset.tar.gz')

filename = '/home/dagutman/dev/KerasSimpsons/simpsons_dataset.tar.gz'

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  #print(data_folders)
  return data_folders
#data_folders = maybe_extract(dest_filename)
data_folders = maybe_extract(filename)


/home/dagutman/dev/KerasSimpsons/simpsons_dataset already present - Skipping extraction of /home/dagutman/dev/KerasSimpsons/simpsons_dataset.tar.gz.


In [3]:
img_width, img_height = 64, 64
train_data_dir = os.sep+os.path.join('home', 'dagutman', 'dev', 'KerasSimpsons',  'simpsons_dataset')
validation_data_dir = os.sep+os.path.join('home', 'dagutman', 'dev', 'KerasSimpsons', 'kaggle_simpson_testset')

nb_train_samples = 30000
nb_validation_samples = 990
epochs = 50
batch_size = 32

In [ ]:
print validation_data_dir

In [ ]:
## need to check the data_folders as I think there are more directories then originally trained on
data_folders

In [4]:
# Model definition
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

    
    
NumLabels = 20
    
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NumLabels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [5]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255.0,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip=True)

# Only rescaling for validation
valid_datagen = ImageDataGenerator(rescale=1. / 255.0)


In [6]:
# Flows the data directly from the directory structure, resizing where needed
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 19548 images belonging to 20 classes.


In [ ]:
### tunablge parameter ===  batch_size is 1
### other one is image size   64 vs 96  vs 128


In [7]:
validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir + "/",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

simpsonsModel = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)


Found 990 images belonging to 20 classes.
Epoch 1/50
937/937 [==============================] - 92s - loss: 2.8107 - acc: 0.1315 - val_loss: 2.6941 - val_acc: 0.1750
Epoch 2/50
937/937 [==============================] - 86s - loss: 2.3187 - acc: 0.2861 - val_loss: 1.9714 - val_acc: 0.3948
Epoch 3/50
937/937 [==============================] - 96s - loss: 1.8123 - acc: 0.4391 - val_loss: 1.6366 - val_acc: 0.4833
Epoch 4/50
937/937 [==============================] - 92s - loss: 1.5397 - acc: 0.5286 - val_loss: 1.2130 - val_acc: 0.6031
Epoch 5/50
937/937 [==============================] - 92s - loss: 1.3522 - acc: 0.5802 - val_loss: 1.0981 - val_acc: 0.6260
Epoch 6/50
937/937 [==============================] - 86s - loss: 1.2414 - acc: 0.6183 - val_loss: 0.9519 - val_acc: 0.6990
Epoch 7/50
937/937 [==============================] - 86s - loss: 1.1370 - acc: 0.6533 - val_loss: 0.9000 - val_acc: 0.6865
Epoch 8/50
937/937 [==============================] - 90s - loss: 1.0634 - acc: 0.6788 - v

In [19]:
import h5py
model.save_weights('simpsons_weights.h5')
# Case-1 I have used like :

# model.compile(loss=’categorical_crossentropy’, optimizer=’Nadam’, metrics=[‘acc’, ‘fmeasure’, ‘precision’, ‘recall’])

# Case-2 and also used :

# def score(yh, pr):
# coords = [np.where(yhh > 0)[0][0] for yhh in yh]
# yh = [yhh[co:] for yhh, co in zip(yh, coords)]
# ypr = [prr[co:] for prr, co in zip(pr, coords)]
# fyh = [c for row in yh for c in row]
# fpr = [c for row in ypr for c in row]
# return fyh, fpr

# pr = model.predict_classes(X_train)
# yh = y_train.argmax(2)
# fyh, fpr = score(yh, pr)
# print ‘Training accuracy:’, accuracy_score(fyh, fpr)
# print ‘Training confusion matrix:’
# print confusion_matrix(fyh, fpr)
# precision_recall_fscore_support(fyh, fpr)

# pr = model.predict_classes(X_test)
# yh = y_test.argmax(2)
# fyh, fpr = score(yh, pr)
# print ‘Testing accuracy:’, accuracy_score(fyh, fpr)
# print ‘Testing confusion matrix:’
# print confusion_matrix(fyh, fpr)
# precision_recall_fscore_support(fyh, fpr)

# What I have observed is that, accuracy of case-1 and case-2 are different?



In [9]:
from sklearn.metrics import confusion_matrix
confusion_matrix()
# >>> y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
# >>> y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
# >>> confusion_matrix(y_true, y_pred, labels=["ant", "bird", "cat"])
# array([[2, 0, 0],
#        [0, 0, 1],
#        [1, 0, 2]])

<function sklearn.metrics.classification.confusion_matrix>

In [12]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as img
import pandas as pd
root_dir = '\\tmp\\simpsons\\internet'

# train_generator.class_indices
chardict = {'grampa': 0,
 'apu': 1,
 'bart': 2,
 'burns': 3,
 'chief': 4,
 'comic': 5,
 'edna': 6,
 'homer': 7,
 'brockman': 8,
 'krusty': 9,
 'lenny': 10,
 'lisa': 11,
 'marge': 12,
 'mayor': 13,
 'milhouse': 14,
 'moe': 15,
 'ned': 16,
 'nelson': 17,
 'skinner': 18,
 'sideshow': 19}

rows = 3
cols = 3
fig, ax = plt.subplots(rows, cols, frameon=False, figsize=(7, 7))
fig.suptitle('Internet images', fontsize=20, y = 1.03)
count=0
for i in range(rows):
    for j in range(cols):
        all_files = os.listdir(root_dir)
        imgpath = os.path.join(root_dir, all_files[count])
        img = Image.open(imgpath)
        img = img.resize((img_width, img_height), Image.ANTIALIAS)
        ax[i][j].imshow(img)
        img = img_to_array(img)
        img = img/255.0
        img = img.reshape((1,) + img.shape)
        pred = model.predict(img, batch_size= 1)
        pred = pd.DataFrame(np.transpose(np.round(pred, decimals = 3)))
        pred = pred.nlargest(n = 3, columns = 0)
        pred['char'] = [list(chardict.keys())[list(chardict.values()).index(x)] for x in pred.index]
        charstr = ''
        for k in range(0,3):
            if k < 2:
                charstr = charstr+str(pred.iloc[k,1])+': '+str(pred.iloc[k,0])+'\n'
            else:
                charstr = charstr+str(pred.iloc[k,1])+': '+str(pred.iloc[k,0])
        ec = (0, .8, .1)
        fc = (0, .9, .2)
        count = count + 1
        ax[i][j].text(0, -10, charstr, size=10, rotation=0,
                ha="left", va="top", 
                bbox=dict(boxstyle="round", ec=ec, fc=fc, alpha = 0.7))
plt.setp(ax, xticks=[], yticks=[])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

ImportError: numpy.core.multiarray failed to import